# Simple Workflow with jobflow

## Define workflow with jobflow

In [1]:
from jobflow import job, Flow

/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
from python_workflow_definition.jobflow import write_workflow_json

In [3]:
from simple_workflow import (
    add_x_and_y as _add_x_and_y, 
    add_x_and_y_and_z as _add_x_and_y_and_z,
)

In [4]:
workflow_json_filename = "jobflow_simple.json"

In [5]:
add_x_and_y = job(_add_x_and_y, data=["x", "y", "z"])
add_x_and_y_and_z = job(_add_x_and_y_and_z)

In [6]:
obj = add_x_and_y(x=1, y=2)

In [7]:
w = add_x_and_y_and_z(x=obj.output.x, y=obj.output.y, z=obj.output.z)

In [8]:
flow = Flow([obj, w])

In [9]:
write_workflow_json(flow=flow, file_name=workflow_json_filename)

In [10]:
!cat {workflow_json_filename}

{"nodes": {"0": "simple_workflow.add_x_and_y", "1": "simple_workflow.add_x_and_y_and_z", "2": 1, "3": 2}, "edges": [{"target": 0, "targetHandle": "x", "source": 2, "sourceHandle": null}, {"target": 0, "targetHandle": "y", "source": 3, "sourceHandle": null}, {"target": 1, "targetHandle": "x", "source": 0, "sourceHandle": "x"}, {"target": 1, "targetHandle": "y", "source": 0, "sourceHandle": "y"}, {"target": 1, "targetHandle": "z", "source": 0, "sourceHandle": "z"}]}

## Load Workflow with aiida

In [11]:
from aiida import load_profile

load_profile()

Profile<uuid='c42e9c9e33c74bf98d1b7929313a2856' name='adis'>

In [12]:
from python_workflow_definition.aiida import load_workflow_json

In [13]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [ ]:
wg.run()

03/22/2025 09:10:55 AM <674> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/22/2025 09:10:55 AM <674> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|continue_workgraph]: tasks ready to run: pickle_node3,pickle_node4
03/22/2025 09:10:55 AM <674> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|run_tasks]: Run task: pickle_node3, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 1}


03/22/2025 09:10:56 AM <674> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|run_tasks]: Run task: pickle_node4, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 2}


03/22/2025 09:10:56 AM <674> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [31|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 39, 47
03/22/2025 09:10:59 AM <674> aiida.scheduler.direct: [WARNING] Unrecognized job_state 'p' for job id Activating
03/22/2025 09:10:59 AM <674> aiida.engine.transports: [ERROR] Exception whilst using transport:
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/engine/transports.py", line 106, in request_transport
    yield transport_request.future
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/engine/processes/calcjobs/manager.py", line 109, in _get_jobs_from_scheduler
    scheduler_response = scheduler.get_jobs(**kwargs)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/aiida/schedulers/plugins/direct.py", line 269, in get_jobs
    job_stats = super().get_jobs(jobs=jobs, user

## Load Workflow with pyiron_base

In [ ]:
from pyiron_base import Project

In [ ]:
from python_workflow_definition.pyiron_base import load_workflow_json

In [ ]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

In [ ]:
delayed_object = load_workflow_json(project=pr, file_name=workflow_json_filename)
delayed_object.draw()

In [ ]:
delayed_object.pull()